In [8]:
# !pip install pip --upgrade
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

import pandas as pd
# !pip install pyopenssl --upgrade
# !pip install --upgrade smart_open
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

# for processing
import re
import nltk
from nltk.tokenize import word_tokenize


from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_colwidth', None)

from datetime import datetime
from nltk.tokenize import RegexpTokenizer

In [2]:
%%time
embeddings_dict_6B_50D = {}
with open("glove.6B/glove.6B.50d.txt", 'r', encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = ' '.join(values[:-50]).lower().strip()
        vector = np.asarray(values[-50:], "float32")
        embeddings_dict_6B_50D[word] = vector
        
def vectorize_text(text):
    vectors = [embeddings_dict_6B_50D.get(word) for word in str(text).split() if word in embeddings_dict_6B_50D]
    vectors = [v for v in vectors if v is not None]  # remove any None values
    if vectors:
        vectorized = np.mean(vectors, axis=0)
    else:
        vectorized = np.zeros(50)  # if there are no vectors, return a zero-vector
    return vectorized

Wall time: 3.95 s


In [3]:
def text_normalizer(text):
    if text:
        # Use NLTK RegexpTokenizer for tokenization. 
        # This tokenizer splits the text by white space and also keeps tokens like "wasn't" and "don't".
        tokenizer = RegexpTokenizer(r'\b\w[\w\'-]*\w\b|\w')
        words = tokenizer.tokenize(text)

        # Clean up any token with repeating characters like '666', 'aaa', '!!!!!!', substitute them with empty string ''.
        # This includes 'XXXX' maskings in the text created by CFPB.
        words = [re.sub(r'(\w)\1{2,}', '', word) if re.search(r'(\w)\1{2,}', word) else word for word in words]

        # Convert to lowercase and remove punctuations.
        words = [word.lower().strip() for word in words]

        # Substitute the tokens with "" where they are just numbers.
        words = ['' if word.isdigit() else word for word in words]

        # Join the words back into a single string.
        text = ' '.join([word for word in words if word])
    
    return text

In [9]:
%%time
cfpb_df = pd.read_csv("CFPB RAW complaints 5.9.2023.csv")

<timed exec>:1: DtypeWarning: Columns (9,16) have mixed types. Specify dtype option on import or set low_memory=False.


Wall time: 18.1 s


In [10]:
%%time
# Convert the 'date' column to datetime format
print("Before droping nan narrative: ", len(cfpb_df))
cfpb_df.dropna(subset=['Consumer complaint narrative'], inplace=True)
cfpb_df['Date received'] = pd.to_datetime(cfpb_df['Date received'])
cfpb_df['narr_len'] = cfpb_df['Consumer complaint narrative'].apply(lambda x:len(str(x)))
cfpb_df['days_to_today'] = (datetime.now().date() - cfpb_df['Date received'].dt.date).dt.days

cfpb_df['narr_len'] = cfpb_df['narr_len'].astype(int)
cfpb_df['days_to_today'] = cfpb_df['days_to_today'].astype(int)

cfpb_df['days_to_today'] = (datetime.now().date() - cfpb_df['Date received'].dt.date).dt.days

cfpb_df['clean_narr'] = cfpb_df['Consumer complaint narrative'].apply(text_normalizer)
cfpb_df['narr_head_vec'] = cfpb_df['clean_narr'].apply(lambda x: vectorize_text(x[:500]))

print("After droping nan narrative: ", len(cfpb_df))

Before droping nan narrative:  3599657
After droping nan narrative:  1300361
Wall time: 5min 25s


In [41]:
# df = cfpb_df[cfpb_df['Complaint ID'].isin([5830282,5830057,5830027,5830056])].copy()#sample(15)
# df

In [42]:
# from scipy.spatial.distance import pdist, squareform
# vectors = df['narr_head_vec'].tolist()

# vectors_2d = np.vstack(vectors)
# dists = pdist(vectors_2d, metric='euclidean')
# dist_matrix = squareform(dists)
# pd.DataFrame(dist_matrix)

In [43]:
# point1 = df['narr_head_vec'].to_list()[0]
# point2 = df['narr_head_vec'].to_list()[1]
# point3 = df['narr_head_vec'].to_list()[2]
# point4 = df['narr_head_vec'].to_list()[3]
# point5 = cfpb_df['narr_head_vec'].sample().to_list()[0]

# print(np.linalg.norm(point1 - point2))
# print(np.linalg.norm(point1 - point3))
# print(np.linalg.norm(point1 - point5))

In [44]:
small_cfpb_df = cfpb_df[['State', 'ZIP code','Complaint ID','narr_len', 'days_to_today','narr_head_vec']].copy()
small_cfpb_df[['State', 'ZIP code']] = small_cfpb_df[['State', 'ZIP code']].fillna('')

In [45]:
# when running group by, the input is a mini dataframe with same ['Product', 'Issue', 'State', 'ZIP code']
def find_duplicate_narr(df):
    small_df = df[['Complaint ID','narr_len', 'days_to_today', 'narr_head_vec']].copy()
    
    def find_dupi_in_small_df(row_narr_len, row_to_day, row_narr_head_vec, small_df):
        tmp_df = small_df.query("narr_len <= @row_narr_len*1.2 & narr_len >= @row_narr_len*0.8 & days_to_today <= @row_to_day+5 & days_to_today >= @row_to_day-5").copy()    
        tmp_df['eclidean_dist'] = tmp_df['narr_head_vec'].apply(lambda x: np.linalg.norm(x - row_narr_head_vec))
        dupli_df = tmp_df[tmp_df['eclidean_dist']<0.25]
        dupli_id_list = sorted(dupli_df['Complaint ID'].to_list())
        return dupli_id_list
    
    df['dupi_id'] = small_df.apply(lambda row: find_dupi_in_small_df(row['narr_len'], int(row['days_to_today']), row['narr_head_vec'], small_df),axis=1)
    
    return df

In [46]:
%%time
small_cfpb_df = small_cfpb_df.groupby(['State', 'ZIP code']).apply(func=find_duplicate_narr)

Wall time: 42min 14s


In [47]:
small_cfpb_df['dupi_len'] = small_cfpb_df['dupi_id'].apply(lambda x: len(x))
small_cfpb_df['dupi_id'] = small_cfpb_df['dupi_id'].apply(lambda x: ";".join([str(y) for y in x]))

In [49]:
merged_df = cfpb_df.merge(small_cfpb_df[['Complaint ID', 'dupi_id', 'dupi_len']], on='Complaint ID', how='left').drop(['narr_head_vec'], axis=1)

In [59]:
dupli_df = merged_df[merged_df.dupi_len>1].sort_values('dupi_id').copy()
dupli_df

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,...,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,narr_len,days_to_today,clean_narr,dupi_id,dupi_len
366585,2015-03-19,Credit reporting,NaN,Incorrect information on credit report,Account status,"I have been disputing incorrect information on my Experian XXXX XXXX XXXX credit report but they are unwilling to allow me to dispute their "" investigation results ''. The creditors are XXXX XXXX and XXXX XXXX XXXX. The report reflects late payments and the information is inaccurate. XXXX creditors have agreed to have the items corrected on all XXXX credit bureaus but only XXXX made the correction.",Company chooses not to provide a public response,Experian Information Solutions Inc.,FL,33157,...,2015-03-19,Closed with explanation,Yes,No,1290842,401,3035,i have been disputing incorrect information on my experian credit report but they are unwilling to allow me to dispute their investigation results the creditors are and the report reflects late payments and the information is inaccurate creditors have agreed to have the items corrected on all credit bureaus but only made the correction,1290842;1290843,2
253477,2015-03-19,Credit reporting,NaN,Incorrect information on credit report,Account status,"I have been disputing incorrect information on my XXXX & Trans Union credit report but they are unwilling to allow me to dispute their "" investigation results ''. The creditors are XXXX XXXX and XXXX XXXX XXXX. The report reflects late payments and the information is inaccurate. Both creditors have agreed to have the items corrected on all XXXX credit bureaus but only XXXX made the correction.",NaN,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",FL,33157,...,2015-03-19,Closed with explanation,Yes,No,1290843,396,3035,i have been disputing incorrect information on my trans union credit report but they are unwilling to allow me to dispute their investigation results the creditors are and the report reflects late payments and the information is inaccurate both creditors have agreed to have the items corrected on all credit bureaus but only made the correction,1290842;1290843,2
46863,2015-03-19,Credit reporting,NaN,Incorrect information on credit report,Information is not mine,there is a XXXX XXXX account on my credit report that does not belong to me. I have attem have attempted to dispute several times to no avail.,NaN,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",FL,33157.0,...,2015-03-25,Closed with non-monetary relief,Yes,No,1290847,142,3035,there is a account on my credit report that does not belong to me i have attem have attempted to dispute several times to no avail,1290847;1290849,2
1122799,2015-03-19,Credit reporting,NaN,Incorrect information on credit report,Information is not mine,there is a XXXX BP account on my credit report that does not belong to me. I have attem have attempted to dispute several times to no avail.,Company chooses not to provide a public response,Experian Information Solutions Inc.,FL,33157.0,...,2015-03-19,Closed with explanation,Yes,No,1290849,140,3035,there is a bp account on my credit report that does not belong to me i have attem have attempted to dispute several times to no avail,1290847;1290849,2
762120,2015-03-19,Debt collection,I do not know,Cont'd attempts collect debt not owed,Debt is not mine,"Call from XXXX XXXX XXXX XXXX XXXX XXXX seeking XXXX XXXX. XXXX purchased bad debt or bad contact information from Northland Group. Call from ( XXXX ) XXXX, I explained debt is not mine. Lady got rude when I refused to answer her questions about me. Told her do not call again. Subsequently received an automated call from ( XXXX ) XXXX seeking XXXX XXXX.",NaN,Northland Group Inc,FL,32256.0,...,2015-03-19,Closed with explanation,Yes,No,1291079,355,3035,call from seeking purchased bad debt or bad contact information from northland group call from i exp

In [76]:
df = dupli_df.groupby(['Product','dupi_id']).sum().reset_index()
df['dupi_id_len'] = df['dupi_id'].apply(lambda x: len(x.split(";")))
df[df['dupi_id_len']!=np.sqrt(df['dupi_len'])].sort_values('days_to_today')

,Product,dupi_id,Complaint ID,narr_len,days_to_today,dupi_len,dupi_id_len
148374,"Credit reporting, credit repair services, or other personal consumer reports",6876195;6876460,6876195,300,79,2,2
148373,"Credit reporting, credit repair services, or other personal consumer reports",6876043;6876460,6876043,400,79,2,2
148372,"Credit reporting, credit repair services, or other personal consumer reports",6876043;6876195;6876460,6876460,400,79,3,3
148345,"Credit reporting, credit repair services, or other personal consumer reports",6803373;6803530,6803373,100,95,2,2
148341,"Credit reporting, credit repair services, or other personal consumer reports",6799272;6799484,6799484,400,96,2,2
...,...,...,...,...,...,...,...
36098,"Credit reporting, credit repair services, or other personal consumer reports",3594857;3594862;3594921;3594923;3594927;3594933;3594953;3594954;3594958;3594960;3594961;3594965;3595013;3595014;3595030;3595035;3595036;3595065;3595069;3595070;3595071;3595072;3595074;3595076;3595103;3595106;3595108;3595110;3595112;3595113;3595146;3595147;3595454;3595459;3595690;3595691,79091790,4400,26178,792,36
17964,"Credit reporting, credit repair services, or other personal consumer reports",3056242;3056257;3056267;3056285;3056286;3056325;3058078;3058081;3058082;3058084;3059126;3059171;3059275;3059298;3059310;3059359;3059360;3059382;3059392;3060236;3060301;3060310;3060336;3060425;3060435;3061514;3061529;3061530;3061534;3061539;3061541;3061544;3061546,55067805,14400,30877,594,33
2640,Credit reporting,1605520;1605526;1605527;1607824;1607826;1607827;1607828;1607830;1607832;1607834;1607836;1607842;1610146;1610148;1610150;1610151;1610153;1610154;1610156;1610161;1610165;1610170;1610171;1610177;1610178;1610180;1610183,28973582,16200,50836,486,27
2638,Credit reporting,1605519;1605521;1605522;1605523;1605524;1605525;1607811;1607825;1607829;1607831;1607837;1607838;1607840;1607841;1610147;1610149;1610152;1610155;1610157;1610158;1610159;1610160;1610162;1610163;1610164;1610166;1610167;1610168;1610169;1610172;1610173;1610174;1610175;1610176;1610179;1610181;1610182;1610184,37033808,20800,64952,874,38


In [78]:
x = "3594857;3594862;3594921;3594923;3594927;3594933;3594953;3594954;3594958;3594960;3594961;3594965;3595013;3595014;3595030;3595035;3595036;3595065;3595069;3595070;3595071;3595072;3595074;3595076;3595103;3595106;3595108;3595110;3595112;3595113;3595146;3595147;3595454;3595459;3595690;3595691"
len(x.split(";"))

36

In [60]:
pivot_table = pd.pivot_table(dupli_df, index='dupi_len', columns='Product', aggfunc=len, fill_value=0)
pivot_table

Company                                            \
Product  Bank account or service Checking or savings account Consumer Loan   
dupi_len                                                                     
2                             82                         333            45   
3                             13                          49            29   
4                              5                          21            14   
5                              2                           3             0   
6                              0                           5             0   
7                              0                           3             3   
8                              0                           3             1   
9                              0                           0             0   
10                             0                           1             0   
11                             0                           0             1   
12                             0                           0             0   
13                             0                           0             0   
14                             0                           0             0   
15                             0                           0             0   
16                             0                           0             0   
17                             0                           0             0   
18                             0                           0             0   
19                             0                           0             0   
20                             0                           0             0   
21                             0                           0             0   
22                             0                           0             0   
23                             0                           0             0   
24                             0                           0             0   
25                             0                           0             0   
26                             0                           0             0   
27                             0                           0             0   
28                             0                           0             0   
29                             0                           0             0   
30                             0                           0             0   
31                             0                           0             0   
32                             0                           0             0   
33                             0                           0             0   
34                             0                           0             0   
35                             0                           0             0   
36                             0                           0             0   
37                             0                           0             0   
38                             0                           0             0   
39                             0                           0             0   
40                             0                           0             0   
41                             0                           0             0   
42                             0                           0             0   
44                             0                           0             0   
45                             0                           0             0   
48                             0                           0             0   
49                             0                           0             0   
51                             0                           0             0   
53                             0                           0             0   
54                             0                           0             0   
55                             0               

In [61]:
dupli_df['narr_len'] = dupli_df['narr_len'].apply(lambda x: round(x, -2))
pivot_table = pd.pivot_table(dupli_df, index='dupi_len', columns='narr_len', aggfunc=len, fill_value=0)
pivot_table

Company                                                          \
narr_len   0      100    200    300    400    500    600    700    800     
dupi_len                                                                   
2           1899  14648  18273  23625  22948  19950  18045  16508  11134   
3            968   6338   8095  11907  11450   9603   8298   8021   5492   
4             88    873   1370   2655   1933   1543   1533   1403    928   
5             59    458    789   1775   1013    767    739    817    425   
6             50    323    401   1067    556    497    421    576    272   
7             21    203    216    852    266    328    224    194    136   
8              0    135    174    590    266    168    170    181    116   
9              0    113    155    543    186    122     94    134     93   
10            10     58    112    397    116    114     84    114     55   
11            33     88     93    362    141     95     91     66     16   
12            12     69     42    314     91     33     44     64     49   
13             0     64     64    294     73     24     42     33      0   
14             0     63     11    131     86     60      3     36      3   
15             0     28     31    163     76     25     67     27     23   
16             0     17     58    323     31     23     32     27      8   
17             0      3     17    165     24     15     12     29     26   
18             0     19     19    162     51     18     26      8      0   
19             0     20     38    201     29      5     20      8     34   
20             0      3     15    220     26      0      0      0     28   
21             0      3      0    123     11      4     11      0      0   
22             0     44     23    192     22      7      2      0      0   
23             0      4      2     98     44      8      0      0      0   
24             0      0      2     99     21      4      0      5      6   
25             0      4      0     46     12     11      0      0      0   
26             0     14      0     76      9     12     26      0      0   
27             0      1      0     92     40      1      3      0      9   
28             0     28      0     41      8      0      0      0      0   
29             0      0      0     48      4      1      0     10      1   
30             0      3      0     24      9      0      0      0      0   
31             0      0      0     75     16     10      5      0      0   
32             0      4      0     98     29      6      9      0      0   
33             0      0      8     80     14      5      0      0     18   
34             0      0      0      0      0      0      0      0      0   
35             0      0      4     17      0      0      0      0      0   
36             0      0     22     28     21      0      0      0      0   
37             0      0      0      3      0      0      0      0      0   
38             0      0      2      3      0     11      0      0      0   
39             0      0      0     48      0      0      0      0      0   
40             0      0      0      0     18     22      0      0      0   
41             0      0      0     47      1      0      0      0      0   
42             0      0      0      0     12      1      2      0      0   
44             0      0      0      0     10      2      1      0      0   
45             0      0      0     45      0      0      0      0      0   
48             0      0      0      0      0      2      0      0      0   
49             0      0      0      0      9      0      0      0      0   
51             0      0      0      0     35     14      0      0      0   
53             0      0      0      0      1     25      2      0      0   
54             0      0      0      0      2      0      0      0      0   
55             0      0      0      0      0      1      0      0      0   
56             0      0      0      0      0      5      

In [63]:
dupli_df[dupli_df.dupi_len>25]

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,...,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,narr_len,days_to_today,clean_narr,dupi_id,dupi_len
379287,2015-08-08,Credit reporting,NaN,Improper use of my credit report,Report improperly shared by CRC,"While checking my personal credit report, I noticed an unauthorized and fraudulent credit inquiry made by XXXX on or about XX/XX/XXXX on Transunion. I did not authorized anyone employed by this company to make any inquiry and view my credit report. XXXX has violated the Fair Credit Reporting Act Section 1681b ( c ). They were not legally entitled to make this fraudulent inquiry. This is a serious breach of my privacy rights. \nI have requested that they mail me a copy of my signed authorization form that gave them the right to view my credit within XXXX ( XXXX ) business days so that I can verify its validity and advised them that if they can not provide me with proof that I authorized them to view my credit report then I am demanding that they contact the credit bureaus immediately and have them remove the unauthorized and fraudulent hard inquiry immediately. I also requested that they remove my personal information from their records. My Social Security # is XXXX and my Date of Birth is XX/XX/XXXX in case it is needed to locate the fraudulent inquiry in their system.",Company chooses not to provide a public response,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",CA,92563.0,...,2015-08-08,Closed with explanation,Yes,No,1509798,1100,2893,while checking my personal credit report i noticed an unauthorized and fraudulent credit inquiry made by on or about xx xx on transunion i did not authorized anyone employed by this company to make any inquiry and view my credit report has violated the fair credit reporting act section 1681b c they were not legally entitled to make this fraudulent inquiry this is a serious breach of my privacy rights i have requested that they mail me a copy of my signed authorization form that gave them the right to view my credit within business days so that i can verify its validity and advised them that if they can not provide me with proof that i authorized them to view my credit report then i am demanding that they contact the credit bureaus immediately and have them remove the unauthorized and fraudulent hard inquiry immediately i also requested that they remove my personal information from their records my social security is and my date of birth is xx xx in case it is needed to locate the fraudulent inquiry in their system,1509792;1509793;1509794;1509796;1509798;1509799;1509801;1509805;1509806;1509807;1509808;1509810;1509811;1509812;1509814;1509815;1509816;1509819;1509820;1510117;1510119;1510126;1510131;1510134;1510136;1510141;1510142;1510145;1510146;1510148;1510149;1510152;1510154;1510157,34
585415,2015-08-09,Credit reporting,NaN,Improper use of my credit report,Report improperly shared by CRC,"While checking my personal credit report, I noticed an unauthorized and fraudulent credit inquiry made by XXXX XXXX XXXX XXXX CO on or about XX/XX/XXXX on Transunion. I did not authorized anyone employed by this company to make any inquiry and view my credit report. XXXX XXXX XXXX XXXX CO has violated the Fair Credit Reporting Act Section 1681b ( c ). They were not legally entitled to make this fraudulent inquiry. This is a serious breach of my privacy rights. \nI have requested that they mail me a copy of my signed authorization form that gave them the right to view my credit within five ( 5 ) business days so that I can verify its validity and advised them that if they can not provide me with proof that I authorized them to view my credit report then I am demanding that they contact the credit bureaus immediately and have them remove the unauthorized and fraudulent hard inquiry immediately. I also requested that they remove my personal 

In [77]:
merged_df.to_csv('CFPB with Duplicate Marked NEW.csv', index=False)